In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
import time

In [2]:
# Define the Filepath for the Dataset
filepath = 'OPENDATA_BOOKING_CALL_A_BIKE.csv'

# Define the Chunksize
chunksize = 10 ** 6

# Define the specific Timerange 
date_after = pd.Timestamp('2013-12-31 23:59:59')
date_before = pd.Timestamp('2016-01-01 00:00:00')

# Define the Columns that contain Dates
dates_from_data = ['DATE_BOOKING',
                   'DATE_FROM',
                   'DATE_UNTIL']

# Define the columns that are needed from the Dataset
needed_cols = ['DATE_BOOKING' ,
               'DATE_FROM',
               'DATE_UNTIL',
               'START_RENTAL_ZONE',
               'END_RENTAL_ZONE',
               'CITY_RENTAL_ZONE']

# Definition that imports the Cataset in Chunks of 1,000,000 rows to not clog the Memory
def load_csv():
    
    data_iterator = pd.read_csv(filepath,
                                chunksize=chunksize,
                                sep=';',
                                parse_dates=dates_from_data,
                                usecols=needed_cols)
    chunk_list = []  

    # Each chunk is in dataframe format
    for data_chunk in data_iterator:  
        # Filter the Data for the City Hamburg and the given Time Horizon
        filtered_chunk = data_chunk[(data_chunk['CITY_RENTAL_ZONE'] == 'Hamburg') &
                                    (data_chunk['DATE_UNTIL'] > date_after) &
                                    (data_chunk['DATE_UNTIL'] < date_before)]
        # Append the chunk into a list, so we have the continued data
        chunk_list.append(filtered_chunk)
    filtered_data = pd.concat(chunk_list)
    return filtered_data

# Time how long it takes to load the Bike Dataset and insert it into the bike_data variable
%time bike_data = load_csv()

# Format the Dates from Object to DateTime Types
bike_data['DATE_BOOKING'] = pd.to_datetime(bike_data['DATE_BOOKING'],
                                                        format='%Y%m%d %H:%M:%S')
bike_data['DATE_FROM'] = pd.to_datetime(bike_data['DATE_FROM'],
                                                        format='%Y%m%d %H:%M:%S')
bike_data['DATE_UNTIL'] = pd.to_datetime(bike_data['DATE_UNTIL'],
                                                        format='%Y%m%d %H:%M:%S')

CPU times: user 1min 37s, sys: 14.2 s, total: 1min 51s
Wall time: 1min 59s


In [3]:
bike_data

,DATE_BOOKING,DATE_FROM,DATE_UNTIL,START_RENTAL_ZONE,END_RENTAL_ZONE,CITY_RENTAL_ZONE
0,2014-01-01 00:34:54,2014-01-01 00:34:54,2014-01-01 00:50:14,U-Bahn Baumwall,Mönckebergstraße / Rosenstraße,Hamburg
1,2014-01-01 01:39:55,2014-01-01 01:39:55,2014-01-01 01:57:27,Bahnhof Altona Ost/Max-Brauer-Allee,Schulterblatt/Eifflerstraße,Hamburg
2,2014-01-01 01:40:20,2014-01-01 01:40:20,2014-01-01 01:53:09,Weidestraße/Biedermannplatz,Jarrestraße / Rambatzweg,Hamburg
3,2014-01-01 01:56:00,2014-01-01 01:56:00,2014-01-01 01:56:41,Große Bergstraße / Jessenstraße,Große Bergstraße / Jessenstraße,Hamburg
4,2014-01-01 02:05:55,2014-01-01 02:05:55,2014-01-01 02:13:49,Mundsburg / Schürbeker Straße,Bartholomäusstraße/Beim Alten Schützenhof,Hamburg
...,...,...,...,...,...,...
8915006,2015-12-31 23:15:08,2015-12-31 23:15:08,2015-12-31 23:36:53,Löwenstraße/Eppendorfer Weg,Schulterblatt/Eifflerstraße,Hamburg
8915007,2015-12-31 23:28:26,2015-12-31 23:28:26,2015-12-31 23:43:06,Lortzingstraße/Friedrichsberger Straße,Eduard-Rhein-Ufer / Schwanenwik,Hamburg
8915076,2015-12-31 22:40:18,2015-12-31 22:40:18,2015-12-31 22:48:34,Hohenzollernring/Friedensallee,Bahnhof Altona West / Busbahnhof,Hamburg
8915078,2015-12-31 23:37:41,2015-12-31 23:37:41,2015-12-31 23:47:13,Burgstraße/Hammer Landstraße,Alsterschwimmhalle/Ifflandstraße,Hamburg


In [4]:
#TEMPERATURE DATA STATIONS FOR 'HAMBURG'
#00954 UFS Deutsche Bucht    -- geoBreite = 54.1796   geoLaenge = 7.4587 This is in the Nordsee and can't be used to the city hamburg
#01228 UFS TW Ems            -- geoBreite = 54.1651   geoLaenge = 6.3460 Same as above
#01975 Hamburg-Fühlsbüttel   -- geoBreite = 53.6332   geoLaenge = 9.9881
#01981 Hamburg-Neuwiedenthal -- geoBreite = 53.4777   geoLaenge = 9.8957
#06254 Hamburg-Lotsenhöft    -- not available
#13904 Nordseeboje 2         -- only till 2010

#TODO GET GOOGLE MAPS COORDINATES WITH PICTURES https://www.gpskoordinaten.de

In [5]:
# Define the filepath for the Temperature Data derived from the German Weather Service Data (DWD)
filepath_temperature_dict = ['temperature_station_01975.csv', 'temperature_station_01981.csv']

# Read the .csv files from the stations 1975 and 1981
temperature_station_1975 = pd.read_csv(filepath_temperature_dict[0], sep=';',
                                       usecols=['STATIONS_ID','MESS_DATUM','TT_TU'])
temperature_station_1981 = pd.read_csv(filepath_temperature_dict[1], sep=';',
                                       usecols=['STATIONS_ID','MESS_DATUM','TT_TU'])

# Rename the columns to clearly know what they stand for
temperature_station_1975.columns = ['ID', 'DATE', 'TEMPERATURE']
temperature_station_1981.columns = ['ID', 'DATE', 'TEMPERATURE']

In [6]:
# Format the Dates from Object to DateTime Types
temperature_station_1975['DATE'] = pd.to_datetime(temperature_station_1975['DATE'],
                                                        format='%Y%m%d%H')
temperature_station_1981['DATE'] = pd.to_datetime(temperature_station_1981['DATE'],
                                                        format='%Y%m%d%H')


# Filter for the Data from 2014-2015
temperature_station_1975 = temperature_station_1975[(temperature_station_1975['DATE'] > date_after) 
                                                    &
                                                    (temperature_station_1975['DATE'] < date_before)]

temperature_station_1981 = temperature_station_1981[(temperature_station_1981['DATE'] > date_after) 
                                                    & 
                                                    (temperature_station_1981['DATE'] < date_before)]

# Reset the Indices so that we start at 0
temperature_station_1975.reset_index(drop=True, inplace=True)
temperature_station_1981.reset_index(drop=True, inplace=True)

#Check if we have all Data from the two Years (2 years * 365 days * 24 hours/day)
if len(temperature_station_1975) == (2 * 365 * 24):
    print('No data missing for station 1975!')
if len(temperature_station_1981) == (2 * 365 * 24):
    print('No data missing for station 1981!')

No data missing for station 1975!
No data missing for station 1981!


In [7]:
# Check for missing Values
print('Absolute values missing for station 1975 are: '
      ,temperature_station_1975['TEMPERATURE'][temperature_station_1975['TEMPERATURE'] == -999.0].count())

print('Absolute values missing for station 1981 are: '
      ,temperature_station_1981['TEMPERATURE'][temperature_station_1981['TEMPERATURE'] == -999.0].count())


Absolute values missing for station 1975 are:  0
Absolute values missing for station 1981 are:  1


In [8]:
# Handle missing Values by just using the other Stations Data at that exact Time 
# (obviously only works if the other Station has all the Data)

index_of_missing_value = temperature_station_1981.index[temperature_station_1981['TEMPERATURE'] ==-999.0]

temperature_station_1981.at[index_of_missing_value, 'TEMPERATURE'] = 
temperature_station_1975.loc[index_of_missing_value].TEMPERATURE

SyntaxError: invalid syntax (<ipython-input-8-82cff36c9884>, line 6)

In [ ]:
# Plot the difference between the two station to check,
# if we should take just one station for simplicity or take a mean of the two
check_station_differences = 
temperature_station_1975['TEMPERATURE'].subtract(temperature_station_1981['TEMPERATURE'])
check_station_differences.plot()

In [ ]:
check_station_differences.describe()

In [ ]:
# The values are too far from each other in their maxima (>5 degrees)
# So we will complete the temperature data by taking the average of each station
temperature_data = pd.DataFrame({'DATE' : temperature_station_1975['DATE'],
                                 'TEMPERATURE': (temperature_station_1975['TEMPERATURE']
                                                 +temperature_station_1975['TEMPERATURE'])/2})

temperature_data['TEMPERATURE'].plot()

In [ ]:
# Precipitation Data

# Define the Filepath for Stations 1975 and 1981
filepath_precipitation_dict = ['precipitation_station_01975.csv','precipitation_station_01981.csv']

# Read the .csv files from the stations 1975 and 1981
precipitation_station_1975 = pd.read_csv(filepath_precipitation_dict[0], sep=';', usecols=[0,1,3])
precipitation_station_1981 = pd.read_csv(filepath_precipitation_dict[1], sep=';', usecols=[0,1,3])

# Rename the columns to clearly know what they stand for
precipitation_station_1975.columns = ['ID','DATE','PRECIPITATION']
precipitation_station_1981.columns = ['ID','DATE','PRECIPITATION']

In [ ]:
#Format the Dates from Object to DateTime Types
precipitation_station_1975['DATE'] = pd.to_datetime(precipitation_station_1975['DATE'], format='%Y%m%d%H')
precipitation_station_1981['DATE'] = pd.to_datetime(precipitation_station_1981['DATE'], format='%Y%m%d%H')

In [ ]:
#Filter for the Data from 2014-2015
precipitation_station_1975 = precipitation_station_1975[(precipitation_station_1975['DATE'] > date_after) 
                                                        &
                                                        (precipitation_station_1975['DATE'] < date_before)]
precipitation_station_1981 = precipitation_station_1981[(precipitation_station_1981['DATE'] > date_after)
                                                        & 
                                                        (precipitation_station_1981['DATE'] < date_before)]

#Reset indices
precipitation_station_1975.reset_index(drop=True, inplace=True)
precipitation_station_1981.reset_index(drop=True, inplace=True)

In [ ]:
# Check for missing values
print('Absolute values missing for station 1975 are: ',
      precipitation_station_1975['PRECIPITATION'][precipitation_station_1975['PRECIPITATION'] == -999.0].count())
print('Absolute values missing for station 1981 are: ',
      precipitation_station_1981['PRECIPITATION'][precipitation_station_1981['PRECIPITATION'] == -999.0].count())


In [ ]:
# Handle missing value by just using the other stations data at that exact time
index_of_missing_value = precipitation_station_1975.index[precipitation_station_1975['PRECIPITATION'] ==-999.0]
value = precipitation_station_1981.loc[index_of_missing_value]['PRECIPITATION']
precipitation_station_1975.at[index_of_missing_value, 'PRECIPITATION'] = value


In [ ]:
# Check if we handled all the missing values

print('Absolute values missing for station 1975 are: ',
      precipitation_station_1975['PRECIPITATION'][precipitation_station_1975['PRECIPITATION'] == -999.0].count())

In [ ]:
# Check the count of the two datasets
print(len(precipitation_station_1975),len(precipitation_station_1981))

In [ ]:
# We need to merge the datasets because both have missing hours (!=17520 values) 
precipitation_merge = pd.merge(precipitation_station_1975, precipitation_station_1981, on='DATE')

In [ ]:
# Define the precipitation DataFrame from the merged data
precipitation_data = pd.DataFrame({'DATE' : precipitation_merge['DATE'], 
                                   'PRECIPITATION' : (precipitation_merge['PRECIPITATION_x']
                                                      +
                                                      precipitation_merge['PRECIPITATION_y'])/2})

In [ ]:
# Plot the precipitation by the hour in mm, so we know what we are dealing with
plt.plot(precipitation_data['DATE'], precipitation_data['PRECIPITATION'])
plt.xlabel('Dates')
plt.ylabel('Precipitation in mm/h')
plt.title('Hourly Precipitation')

In [ ]:
bike_data.index = bike_data.DATE_BOOKING
monthly_data_bike = []

for i in range(1,13):
    monthly_data_bike.append([i,bike_data['DATE_BOOKING'].loc[(bike_data['DATE_BOOKING'].dt.month) == i].count()])

monthly_data_bike = pd.DataFrame(monthly_data_bike)

In [ ]:
monthly_data_bike.info()

# Bike Bookings per Month

In [ ]:
plt.bar(monthly_data_bike[0],monthly_data_bike[1])

In [ ]:
dayofweek_data_bike = []

for i in range(0,7):
    dayofweek_data_bike.append([i,bike_data['DATE_BOOKING'].loc[(bike_data['DATE_BOOKING'].dt.dayofweek) == i].count()])

dayofweek_data_bike = pd.DataFrame(dayofweek_data_bike)


In [ ]:
dayofweek_data_bike.info()

In [ ]:
dayofweek_data_bike[1].cumsum()

# Bike bookings per day of the week

In [ ]:
plt.bar(dayofweek_data_bike[0],dayofweek_data_bike[1])

In [ ]:
hourofday_data_bike = []

for i in range(0,24):
    hourofday_data_bike.append([i,bike_data['DATE_BOOKING'].loc[(bike_data['DATE_BOOKING'].dt.hour) == i].count()])

hourofday_data_bike = pd.DataFrame(hourofday_data_bike)



In [ ]:
hourofday_data_bike

# Bike bookings per hour of day

In [ ]:
plt.bar(hourofday_data_bike[0],hourofday_data_bike[1])

In [ ]:
bike_data['TIME_BOOKED'] = bike_data['DATE_UNTIL'] - bike_data['DATE_FROM']

In [ ]:
bike_data.TIME_BOOKED.describe()

In [ ]:
time_booked = bike_data.TIME_BOOKED
time_booked.index

In [ ]:
time_booked

In [ ]:
hourofday_data_time = []

def integer_to_time_hour(a):
    if a < 10:
        output = str('0'+str(a)+':00')
    else:
        output = str(str(a)+':00')
    return output
    
for i in range(0,24):
    hourofday_data_time.append([integer_to_time_hour(i),time_booked.loc[(bike_data['DATE_BOOKING'].dt.hour) == i].mean()/ datetime.timedelta(minutes=1)])

hourofday_data_time = pd.DataFrame(hourofday_data_time)

In [ ]:
hourofday_data_time

# Average length of booking in minutes per hour of day

In [ ]:
plt.bar(hourofday_data_time.index,hourofday_data_time[1])

In [ ]:
dayofweek_data_time = []

for i in range(0,7):
    dayofweek_data_time.append([i,time_booked.loc[(bike_data['DATE_BOOKING'].dt.dayofweek) == i].mean()
                                / datetime.timedelta(minutes=1)])

dayofweek_data_time = pd.DataFrame(dayofweek_data_time)

In [ ]:
dayofweek_data_time[2] = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
dayofweek_data_time

# Average length of booking per day of week

In [ ]:
plt.bar(dayofweek_data_time[2],dayofweek_data_time[1])

In [ ]:
monthly_data_time = []

for i in range(1,13):
    monthly_data_time.append([i,time_booked.loc[(bike_data['DATE_BOOKING'].dt.month) == i].mean()
                             / datetime.timedelta(minutes=1)])

monthly_data_time = pd.DataFrame(monthly_data_time)

monthly_data_time

In [ ]:
# Average length of booking per month

In [ ]:
plt.bar(monthly_data_time[0],monthly_data_time[1])

In [ ]:
# Mean start time day of week

In [ ]:
time_booked

# Average Booking Start Time per Week (in hours)

In [ ]:
dayofweek_mean = []
daysofweek = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

for i in range(0,7):
    dayofweek_mean.append(bike_data['DATE_FROM'].loc[(bike_data['DATE_FROM'].dt.dayofweek) == i])

dayofweek_averages = []
avg = 0

for i in range(0,7):
    for elem in dayofweek_mean[i]:
        avg += (elem.hour*3600+elem.minute*60+elem.second)
    dayofweek_averages.append(avg/len(dayofweek_mean[i])/60/60)
    avg = 0

dayofweek_averages = pd.Series(dayofweek_averages)

plt.bar(daysofweek, dayofweek_averages)
plt.axis([-0.5,6.5,13.5, 15])
plt.title('Average start time per week')
plt.show()

# Average Start Time per Month (in hours)

In [ ]:
monthly_mean = []
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for i in range(1,13):
    monthly_mean.append(bike_data['DATE_FROM'].loc[(bike_data['DATE_FROM'].dt.month) == i])

monthly_averages = []
avg = 0

for i in range(0,12):
    for elem in monthly_mean[i]:
        avg += (elem.hour*3600+elem.minute*60+elem.second)
    monthly_averages.append(avg/len(monthly_mean[i])/60/60)
    avg = 0

monthly_averages = pd.Series(monthly_averages)

plt.bar(months, monthly_averages)
plt.axis([-0.5,12,13.5, 15])
plt.title('Average start time per month')
plt.show()

In [ ]:
data = pd.DataFrame({'DATE' : pd.date_range(start='1/1/2014', end='31/12/2015 23:00:00', freq='H')})

In [ ]:
data.index = data.DATE
temperature_data.index = temperature_data['DATE']
precipitation_data.index = precipitation_data['DATE']

In [ ]:
data['TEMPERATURE'] = temperature_data['TEMPERATURE']
data['PRECIPITATION'] = precipitation_data['PRECIPITATION']
data['BOOKING_COUNT'] = hourly_bike_data

In [ ]:
data